In [6]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm

data = pd.read_csv("microdata.csv")

data['c_birth_year']=pd.to_datetime(data['c_birth_date']).dt.year


data.drop(columns=['c_birth_date', 'marr_date', 'gr_m_community'], inplace=True)

In [14]:
data_model_a = data[['m_annual_salary_tm2', 'gr_m_family_status']][data['m_annual_salary_tm2']>0]

data_model_a['gr_m_family_status']=np.array([status - 1 for status in data_model_a['gr_m_family_status']])

In [15]:
# Initialize the fixed effects only model
model = bmb.Model('gr_m_family_status ~ m_annual_salary_tm2',
                  data_model_a,
                  family="categorical")

# Get model description
print(model)

# Fit the model using 1000 on each chain
results = model.fit(draws=1000)

# Key summary and diagnostic info on the model parameters
az.summary(results)

# Use ArviZ to plot the results
az.plot_trace(results)

IndexError: tuple index out of range